In [6]:
import pandas as pd
from rtree import index
from shapely.geometry import LineString, Point
import geopandas as gpd
import ezdxf

In [7]:
# Reads points from a .csv file
def read_points_csv(file_path):
    points_df = pd.read_csv(file_path)
    
    # Convert each row in the DataFrame to a Point object
    points = [Point(row['x'], row['y']) for _, row in points_df.iterrows()]
    return points_df, points

# Reads a polyline from a .csv file
def csv_to_polyline(file_path):
    df = pd.read_csv(file_path)
    polyline = [Point(row['x'], row['y']) for _, row in df.iterrows()]
    
    return polyline

# Reads a polyline from a shapefile
def read_shapefile(file_path):
    gdf = gpd.read_file(file_path)
    polyline = gdf.geometry.iloc[0]
    return polyline

# Reads a polyline from a cad file (.dxf or .dwg)
def read_cad_file(file_path):
    doc = ezdxf.readfile(file_path)
    msp = doc.modelspace()

    polyline_entities = [e for e in msp if e.dxftype() == 'LWPOLYLINE']
    
    if not polyline_entities:
        raise ValueError("No LWPOLYLINE found in the CAD file.")
    
    polyline_coords = polyline_entities[0].get_points()
    polyline = [Point(x, y) for x, y in polyline_coords]
    
    return polyline

In [8]:
# Build an R-tree spatial index for efficient nearest segment search.
def build_rtree(polyline):
    idx = index.Index()
    # Iterate through each segment in the polyline.
    for i, segment in enumerate(zip(polyline[:-1], polyline[1:])):
        # Calculate the bounding box of the segment.
        minx, miny = min(segment[0].x, segment[1].x), min(segment[0].y, segment[1].y)
        maxx, maxy = max(segment[0].x, segment[1].x), max(segment[0].y, segment[1].y)
        
        # Insert the bounding box into the R-tree index.
        idx.insert(i, (minx, miny, maxx, maxy))
        
    return idx

# Find the nearest segment of the polyline to the given point using the R-tree index.
def nearest_segment_rtree(point, polyline, idx):
    # Query the R-tree index for the nearest segments to the point.
    nearest_segments = list(idx.nearest((point.x, point.y, point.x, point.y)))
    min_distance = float("inf")
    closest_segment = None

    # Iterate through the nearest_segments to find the one with the minimum distance to the point.
    for segment_index in nearest_segments:
        segment = LineString([polyline[segment_index], polyline[segment_index + 1]])
        distance = segment.distance(point)
        if distance < min_distance:
            min_distance = distance
            closest_segment = segment

    return closest_segment, min_distance

# Calculate the chainage (distance along the polyline) and offset (distance from the polyline) of the point.
def chainage_and_offset(point, polyline, idx):
    # Find the nearest segment to the point and its distance (offset).
    closest_segment, offset = nearest_segment_rtree(point, polyline, idx)
    
    # Calculate the chainage by projecting the point onto the closest segment.
    chainage = closest_segment.project(point)
    
    return chainage, offset

In [9]:
def chain_obtain(polyline, input_file):
    # Read points from a CSV file
    points_df, points = read_points_csv(input_file)  # Unpack the returned tuple

    # Build the R-tree index for the polyline
    rtree_idx = build_rtree(polyline)

    # Initialize lists to store chainage and offset values
    chainage_list = []
    offset_list = []

    # Calculate chainage and offset for each point using the R-tree index
    for point in points:
        chainage, offset = chainage_and_offset(point, polyline, rtree_idx)
        chainage_list.append(chainage)
        offset_list.append(offset)

    # Add chainage and offset values to the points DataFrame
    points_df['chainage'] = chainage_list
    points_df['offset'] = offset_list

    # Export the modified DataFrame with chainage and offset values to a CSV file
    return points_df

In [12]:
# Define the polyline as a list of Point objects
polyline = csv_to_polyline('test_poly.csv')

points_df = chain_obtain(polyline, 'test_points.csv')

AttributeError: 'tuple' object has no attribute 'x'